In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [3]:
df =pd.read_csv(r"C:\Users\pooja kuniyal\Downloads\Feature Engenering files\covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test =train_test_split(df.drop('has_covid', axis=1),
                                                 df['has_covid'], test_size=0.2)
X_train.head()

,age,gender,fever,cough,city
64,42,Male,104.0,Mild,Mumbai
65,69,Female,102.0,Mild,Bangalore
25,23,Male,NaN,Mild,Mumbai
22,71,Female,98.0,Strong,Kolkata
18,64,Female,98.0,Mild,Bangalore


##### without using columnTransformer

In [6]:
# adding simple imputer to fever column
si =SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
# Expected 2D array, got 1D array instead: sol [[ ]] as above or reshape
# also the test data
X_test_fever = si.transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [7]:
X_train_fever

array([[104.        ],
       [102.        ],
       [100.76388889],
       [ 98.        ],
       [ 98.        ],
       [ 99.        ],
       [ 99.        ],
       [104.        ],
       [ 98.        ],
       [101.        ],
       [102.        ],
       [101.        ],
       [ 98.        ],
       [100.76388889],
       [101.        ],
       [100.        ],
       [ 98.        ],
       [101.        ],
       [101.        ],
       [101.        ],
       [ 98.        ],
       [104.        ],
       [100.76388889],
       [100.76388889],
       [101.        ],
       [103.        ],
       [101.        ],
       [104.        ],
       [ 98.        ],
       [101.        ],
       [ 99.        ],
       [104.        ],
       [ 98.        ],
       [104.        ],
       [101.        ],
       [104.        ],
       [102.        ],
       [ 99.        ],
       [104.        ],
       [103.        ],
       [ 98.        ],
       [ 99.        ],
       [ 99.        ],
       [100

In [8]:
# Ordinalencoding -> cough
oe =OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [9]:
X_train_cough[0:9]

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.]])

In [10]:
# OneHotEncoding -> gender,city
ohe =OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city = ohe.transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [11]:
df.city.unique() # 4-1 =3 nd gender 2-1 =1 total =3+1 =4 columns (drop kiya isliye)

array(['Kolkata', 'Delhi', 'Mumbai', 'Bangalore'], dtype=object)

In [12]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [13]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever,X_train_gender_city,
                                     X_train_cough), axis=1)

X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city,
                                    X_test_cough), axis=1)
X_test_transformed.shape

(20, 7)

In [14]:
X_train_cough.shape, X_train_age.shape, X_train_gender_city.shape, X_train_fever.shape

((80, 1), (80, 1), (80, 4), (80, 1))

In [15]:
from sklearn.compose import ColumnTransformer

transformer =ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[["Mild",'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder( sparse_output=False, drop='first'),['gender','city'])
], remainder='passthrough')

In [16]:
transformer.fit_transform(X_train).shape

(80, 7)

In [17]:
transformer.transform(X_test).shape

(20, 7)

**Gist** : agar aap pipeline use nahi kartey ho toh jo series of steps aap taining karne k time use kartey ho, woh same series of steps aapko durabra se likhne padhtey h production code m<br>
problem: agar apne thodi bhi changes kiye h traning wale m toh production m bhi same karna padhta h, not an efficient way of writing code that is why we use pipeline 

# Titanic without using pipeline

In [61]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [62]:
df = pd.read_csv(r"C:\Users\pooja kuniyal\Downloads\Feature Engenering files\train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [63]:
df.drop(columns=['PassengerId', 'Name','Ticket','Cabin'], inplace=True)

In [64]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [65]:
X_train, X_test, y_train,y_test = train_test_split(df.drop(columns='Survived'), df['Survived'],
                                                  test_size=0.2, random_state=42)

In [66]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [67]:
y_train.head(3)

331    0
733    0
382    0
Name: Survived, dtype: int64

In [68]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [69]:
# Applying imputation
si_age =SimpleImputer()
si_embarked =SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [70]:
X_train_embarked[0:10]

array([['S'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['C'],
       ['S'],
       ['S']], dtype=object)

In [71]:
X_train_age[0:10]

array([[45.5       ],
       [23.        ],
       [32.        ],
       [26.        ],
       [ 6.        ],
       [24.        ],
       [45.        ],
       [29.        ],
       [29.49884615],
       [29.49884615]])

In [72]:
# one hot encoding Sex and Embarked
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked =OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# embarked m missing values h islye dono ko agal se OHE kiya h
# jo missing value ki problem humne solve ki woh ek new array m solve hua X_train_embark , original m  change nahi huas

X_train_sex =ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)


In [73]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [74]:
X_train_sex # 2 columns M, F

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [75]:
# 3 alagal array bane h :sex ,age ,embark k liye

X_train_remainder = X_train.drop(columns=['Sex','Age','Embarked'])

In [76]:
X_test_remainder = X_test.drop(columns=['Sex','Age','Embarked'])

In [77]:
X_train_transformed = np.concatenate((X_train_remainder, X_train_age,X_train_sex,X_train_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_remainder, X_test_age, X_test_sex, X_test_embarked), axis=1)

In [78]:
X_test_transformed.shape

(179, 10)

In [79]:
clf =DecisionTreeClassifier()
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [80]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [81]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7877094972067039

In [82]:
# model ko dusri jagah lekar jana h eg. web development k liye
# export kartey ho -pickle library use kar k
#3 models export kar rahe h


# to deal with this error  No such file or directory: 'models/ohe_sex.pkl'
# make directory

import os
import pickle
filename = 'models/ohe_sex.pkl'
filename1 = 'models/ohe_embarked.pkl'
filename2 = 'models/clf.pkl'

os.makedirs(os.path.dirname(filename), exist_ok=True)

os.makedirs(os.path.dirname(filename1), exist_ok=True)

os.makedirs(os.path.dirname(filename2), exist_ok=True)

pickle.dump(ohe_sex, open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf, open('models/clf.pkl','wb'))

# clf (clf, open)== m apne classfier object ko export kar rahi hu is location par
# baki 2 object ohe.sex .. ko bhi send kar rahe h

# Titani using Pipeline

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier


In [3]:
df =pd.read_csv(r"C:\Users\pooja kuniyal\Downloads\Feature Engenering files\train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [7]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
], remainder='passthrough')

#### jab app pipeline build kar rahe ho toh rather than calling by name , call by their index value
above simpleimputer m age ka name nahi diya instead uska index value rakha h,
name nahi likha bcoz problem hoti h, trf1 se jo *output nikalega* woh dataframe nahi , **numpy array** hota h
aur numpy array m columns ka name nahi hota, so if name diya toh age ka code fail ho jayega


In [8]:
# one hot encoding
trf2 =ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
], remainder='passthrough')

original =7 , Ohe k bad (2 col gaya 5 aaya=3) total 7 +3 =10

In [9]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))
])

feature selection kartey time minmaxscaler use karna padhega so standard sacler nahi use kiya

In [10]:
# Feature selection
trf4 = SelectKBest(score_func=chi2, k=8)

In [11]:
# train the model
trf5 = DecisionTreeClassifier()

### create pipeline

In [12]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

# pipeline vs makepipeline
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [96]:
# Alternate syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [13]:
# train
fitting = pipe.fit(X_train, y_train)
fitting

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000021AD4B7AFC0>)),
                ('trf5', DecisionTreeClassifier())])

In [14]:
fitting.get_feature_names_out()

AttributeError: Estimator trf5 does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

In [98]:
# display pipeline

from sklearn import set_config
set_config(display='diagram')

## explore pipeline

In [112]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000001DA4DB58E00>),
 'trf5': DecisionTreeClassifier()}

In [114]:
pipe.named_steps['trf1']  # dictionary se values access kar rahe h

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [115]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [119]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_ # its a list so remove 1st one n then mean

array([29.49884615])

In [121]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_  # most frequent value

array(['S'], dtype=object)

agar appne abhi algorith nahi lagaya hota , toh appka jo pipe bantey
waha par app fit function call nahi kartey, app fit_transform use kartey, kuki 
app sirf data preprocessing kar rahe ho model training nahi

In [101]:
y_pred = pipe.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [102]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.6256983240223464

# Cross Validation using Pipeline

In [103]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train ,y_train,cv=5, scoring='accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [106]:
params ={
    'trf5__max_depth' :[1,2,3,4,5,None]
}

In [107]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV( pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001DA4DB58E00>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [108]:
grid.best_score_

0.6391214419383433

In [109]:
grid.best_params_

{'trf5__max_depth': 2}

# Exporting the Pipeline

In [110]:
import pickle
pickle.dump(pipe, open('pipe.pkl','wb'))